# Coverage tests for collected tweets

Estimate what percentage of Dutch tweets are collected by twiqs.nl

In [1]:
import math
import os
import pandas as pd
import re
import sys
from IPython.display import clear_output

In [2]:
DATADIR = "/home/erikt/projects/puregome/data/text/"

In [3]:
def squeal(text=None):
    clear_output(wait=True)
    if not text is None: print(text)

In [4]:
IDSTR = "id_str"
INREPLYTOSTATUSIDSTR = "in_reply_to_status_id_str"

def getReplyPercentage(filePattern,filePatternReference):
    files = sorted(os.listdir(DATADIR))
    targetFiles = {}
    counter = 0
    for inFileName in files:
        if re.search(filePattern,inFileName):
            counter += 1
            df = pd.read_csv(DATADIR+inFileName,dtype={INREPLYTOSTATUSIDSTR:object})
            counter += len(df)
            for idStr in df[INREPLYTOSTATUSIDSTR]:
                if type(idStr) == type("abc"):
                    targetFiles[idStr] = False
    for inFileName in files:
        if re.search(filePatternReference,inFileName):
            df = pd.read_csv(DATADIR+inFileName,dtype={IDSTR:object})
            for idStr in df[IDSTR]: 
                if idStr in targetFiles: targetFiles[idStr] = True
            squeal(inFileName)
    squeal("")
    return(len([x for x in targetFiles if targetFiles[x]]),len(targetFiles),counter)

In [ ]:
filePattern = "20200609"
filePatternReference = "202006"
lenPart,lenAll,counter = getReplyPercentage(filePattern,filePatternReference)
print("target date: {0}; reference: {1}; percentage: {2}%; total count: {3}".format(filePattern,filePatternReference,\
                                                                                   round(100*lenPart/lenAll,1),counter))

In [ ]:
filePattern = "20200610"
lenPart,lenAll,counter = getReplyPercentage(filePattern,filePatternReference)
print("target date: {0}; reference: {1}; percentage: {2}%; total count: {3}".format(filePattern,filePatternReference,\
                                                                                   round(100*lenPart/lenAll,1),counter))

In [ ]:
filePattern = "20200611"
lenPart,lenAll,counter = getReplyPercentage(filePattern,filePatternReference)
print("target date: {0}; reference: {1}; percentage: {2}%; total count: {3}".format(filePattern,filePatternReference,\
                                                                                   round(100*lenPart/lenAll,1),counter))

## Find best keywords

In [4]:
from nltk.tokenize import TweetTokenizer

In [23]:
FILEPATTERN = "202005|202006"
TEXT = "text"
BASELEXICON = ()
MAXLEXICONSIZE = 800
ALLCOUNT = 596893 # 5623571 # 22798953 # 22798953
LEFT200 = 61599
LEFT100 = 80258 # 349598 # 1724488 # 1231745
LEFT200100 = 43622 # 179089 # 801171 # 732526
LEFT300200100 = 34583 # 130887 # 557205 # 531315
LEFT400300200100 = 29528 # 109099 # 451231 # 439565
LEFT500400300200100 = 0 # # 394609 # 387447
LEFT600500400300200100 = 0 # # 357003 # 352026
LEFT700600500400300200100 = 0 # # 329054 # 324628
LEFT800700600500400300200100 = 0 # # 306745 # 303422

def cleanupText(text):
    text = re.sub(r"\\n"," ",text.lower())
    text = re.sub(r"[#@]"," ",text)
    return(text)

def tokenize(tokenizer,text):
    tokens = tokenizer.tokenize(text)
    return(" ".join(sorted(list(set(tokens)))))

def getTweets():
    files = sorted(os.listdir(DATADIR))
    tokenizer = TweetTokenizer()
    tweets = {}
    for inFileName in files:
        if re.search(FILEPATTERN,inFileName):
            squeal(inFileName)
            df = pd.read_csv(DATADIR+inFileName)
            for text in df[TEXT]:
                text = cleanupText(text)
                text = tokenize(tokenizer,text)
                tweets[text] = True
    return(tweets)

tweets = getTweets()

lexicon = list(BASELEXICON)
processedTweetCounts = []
while len(lexicon) < MAXLEXICONSIZE:
    frequencies = {}
    processedTweetCount = 0
    counter = 0
    toBeDeleted = []
    for text in tweets.keys():
        counter += 1
        if counter%10000 == 0: squeal(str(counter)+"/"+str(len(tweets))+" "+str(len(lexicon)))
        tokens = text.split()
        if len(set(lexicon).intersection(tokens)) > 1:
            toBeDeleted.append(text)
        else:
            processedTweetCount += 1
            for token in tokens:
                if not token in lexicon:
                    if token in frequencies: frequencies[token] += 1
                    else: frequencies[token] = 1
    squeal("deleting preselected tweets...")
    for text in toBeDeleted:
        del(tweets[text])
    squeal(str(processedTweetCount))
    processedTweetCounts.append(processedTweetCount)
    sortedFrequencies = {k:frequencies[k] for k in sorted(frequencies.keys(),key=lambda k:frequencies[k],reverse=True)}
    topToken = [k for k in sortedFrequencies.keys() if re.search(r"[a-z0-9]",k) or \
                                                       (len(k) > 1 and k != ".." and k != "..." or
                                                       (len(k) == 1 and ord(k) >= 127000 and ord(k) < 130000))][0]
    lexicon.append(topToken) 

1403909


In [35]:
outFile = open("tmp.txt","w")
print("track=",end="",file=outFile)
print(",".join([lexicon[i] for i in range(0,len(lexicon)) if i % 2 == 0]),file=outFile)
print("track=",end="",file=outFile)
print(",".join([lexicon[i] for i in range(0,len(lexicon)) if i % 2 != 0]),file=outFile)
outFile.close()
print(" ".join([lexicon[i] for i in range(0,len(lexicon)) if i % 2 == 0]))
print(" ".join([lexicon[i] for i in range(0,len(lexicon)) if i % 2 != 0]))

de een en je van op voor ook die zijn 1 wel nog zo 😂 geen aan 2 mijn naar ze mooi me omg via of er heeft nieuwe al goedemorgen over vacature 👍 🤣 3 eens ameen 😘 kan p dus gewoon ga gaat 😍 i mij morgen 😉 rit leuk fijne open 2020 waar jullie hij 😊 5 sterkte mee he 🏻 vandaag tweet p2000 yes o waarom gaan zeg wtf jaar dankjewel even hahahahaha my werk 7 tijd nooit daar alles e allemaal politie nie m nou sinds amsterdam heerlijk x haar terug inderdaad den maam dm 0 10 graag 😔 im goede komen helemaal 🙏 a1 uur gelukkig goeie 9 foto queen eerste zie geweldig fijn beetje hebben bijna bro 😎 onder k niemand 🤔 lol beter help 🏼 update 🔥 joonie bezig lief nieuw hahahahah baby leuke mag 19 km gezellig 12 eindelijk kom 18 iedereen rotterdam zegt 🥳 <3 teen 01 wauw by hem 👇 🙄 💕 rosé iemand mood idee sorry denk dankje this uw le thierrybaudet 😋 halen nemen zal ingeborgvraagt snel vester maken xxx blonde donde goeiemorgen ma 😀 rivm mn 🙈 at volgende xd thuis 🎉 😜 gezien ongeval go tuurlijk youtube maap ka 10

In [30]:
tweets = getTweets()

20200619-10.out.gz


In [32]:
tweetsCopy = tweets
for i in range(0,len(lexicon)):
    if i % 100 == 0: print(i,len(tweetsCopy))
    toBeDeleted = []
    for text in tweetsCopy.keys():
        tokens = text.split()
        if len(set([lexicon[i]]).intersection(tokens)) > 0:
            toBeDeleted.append(text)
    for text in toBeDeleted:
        del(tweetsCopy[text])

0 22798953
100 1313071
200 769719
300 559828
400 462469
500 406252
600 368855
700 341173


In [33]:
print(i,len(tweetsCopy))

799 318973


In [64]:
print(" ".join([str(processedTweetCounts[i]) for i in range(0,800,100)]))

22798953 1231745 732526 531315 439565 387447 352026 324628


In [45]:
sortedFrequencies = {k:frequencies[k] for k in sorted(frequencies.keys(),key=lambda k:frequencies[k],reverse=True)}
topTokens800 = [k for k in sortedFrequencies.keys() if re.search(r"[a-z0-9]",k) or \
                                                       (len(k) > 1 and k != ".." and k != "..." or
                                                       (len(k) == 1 and ord(k) >= 127000 and ord(k) < 130000))][0:100]

In [46]:
print(",".join(topTokens100))
print(",".join(topTokens200))
print(",".join(topTokens300))
print(",".join(topTokens400))
print(",".join(topTokens500))
print(",".join(topTokens600))
print(",".join(topTokens700))
print(",".join(topTokens800))

de,het,en,een,is,ik,van,in,dat,niet,je,op,voor,met,die,maar,te,ook,zijn,er,als,dan,wat,aan,wel,nog,om,dit,ze,zo,bij,geen,of,nu,al,rt,we,kan,over,heb,naar,1,meer,weer,mijn,deze,heeft,uit,was,mensen,door,hebben,ben,toch,echt,dus,goed,moet,ja,gaat,worden,veel,daar,gaan,hij,hoe,me,hier,mij,eens,wordt,doen,zou,2,kunnen,jij,waar,gewoon,heel,wil,😂,alleen,via,mee,u,jaar,moeten,iets,tot,dag,hun,ga,even,tegen,weet,had,na,ons,maken,jullie
omg,lekker,mooi,vacature,ameen,goedemorgen,😭,hahaha,nieuwe,😘,👍,mooie,haha,a,gefeliciteerd,😍,hahahaha,i,hoor,morgen,oh,3,🥺,s,ok,🤣,rit,open,fijne,tweet,zeker,🏻,p2000,dankjewel,amsterdam,😊,o,leuk,the,sterkte,man,c,a1,yes,and,2020,😉,2020-06-,top,slaap,hahahahaha,to,wie,4,n,5,e,m,werk,gedaan,so,wtf,no,20,0000,stop,my,🥰,maam,h,mm,km,nee,wind,0,he,vandaag,super,best,🙏,graag,hi,😁,7,foto,weg,tijd,x,altijd,🤗,im,you,a2,10,woof,zeg,prachtig,on,🍀,onze
waarom,politie,inderdaad,71,hahahah,:(,heerlijk,klopt,nie,kangen,tz,zonder,allemaal,oof,letsel,vester,pfft,nooit,komen,persin

In [16]:
for token in topTokens100+topTokens200+topTokens300+topTokens400: print(token,end=",")
print()

de,het,en,een,is,ik,van,dat,in,niet,je,op,voor,die,maar,met,te,ook,zijn,er,als,dan,wat,aan,dit,om,wel,nog,ze,zo,bij,geen,nu,over,rt,of,al,we,kan,naar,meer,mensen,heb,1,deze,heeft,was,weer,door,uit,mijn,hebben,ben,toch,moet,dus,echt,goed,worden,ja,gaat,daar,veel,gaan,hier,hij,hoe,wordt,eens,me,mij,doen,jij,zou,waar,kunnen,gewoon,2,heel,tegen,wil,alleen,mee,hun,moeten,u,via,iets,had,😂,tot,ga,jullie,even,weet,maken,alle,dag,na,jaar,lekker,vacature,mooi,goedemorgen,omg,nieuwe,gefeliciteerd,a,😘,👍,mooie,😭,morgen,hoor,open,tweet,😍,zeker,o,p2000,rit,and,fijne,🤣,🏻,slaap,sterkte,amsterdam,leuk,the,😊,n,3,s,to,2020-06-,0000,dankjewel,werk,hahaha,wie,gedaan,10,😉,so,i,haha,2020,🥺,man,a1,c,nee,best,he,11,5,graag,vandaag,oh,4,weg,m,wind,mm,tijd,altijd,top,hahahaha,🥰,foto,h,goede,allemaal,e,0,km,uur,zeg,🍀,onze,nie,🤗,letsel,goeie,20,😁,prachtig,komen,🔴,🙏,no,tz,71,nou,my,den,12,8,💋,waarom,woof,zonder,inderdaad,af,klopt,nooit,ok,x,heerlijk,alles,teen,gij,haar,terug,kijk,xxx,niemand,telegraaf,nos,helemaal,h